In [49]:
import sys
import re
from nltk import sent_tokenize, word_tokenize, pos_tag, ne_chunk
import nltk.data
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
import pickle
from collections.abc import Iterable
import random
from lxml import etree
import gzip
from datetime import datetime as dt

from nltk.tag import ClassifierBasedTagger
from nltk.classify.scikitlearn import SklearnClassifier
# Classifiers
from sklearn.svm import LinearSVC, NuSVC, SVC
from nltk.classify import NaiveBayesClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,\
GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB


from nltk.chunk import ChunkParserI
import unidecode
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn import metrics
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import matplotlib 
matplotlib.use('Agg')
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2
from unwiki import unwiki
from ner.chunker import NamedEntityChunker
import ner

In [3]:
%%script echo opens the latest wikipedia defs
# The results for the search for definition (currently just Wikipedia)
with open('/media/hd1/wikipedia/wiki_definitions_improved.txt', 'r') as wiki_f:
    wikilines = wiki_f.readlines()

opens the latest wikipedia defs


In [4]:
%%script echo Save and create xml zipped file
root = etree.Element('root')

for n, line in enumerate(wikilines):
    title, section, defin = line.split('-#-%-')
    defin = unwiki.loads(eval(defin)).replace('\n', ' ')
    def_tree = etree.Element('definition')
    stmnt = etree.SubElement(def_tree, 'stmnt')
    stmnt.text = unidecode.unidecode(defin)
    dfndum = etree.SubElement(def_tree, 'dfndum')
    dfndum.text = title.strip()
    if stmnt.text != '':
        root.append(def_tree)
    #if n > 4976:
    #    break
#print(etree.tostring(root, pretty_print=True).decode('utf8')) 

with gzip.open('/media/hd1/wikipedia/wiki_definitions_improved.xml.gz', 'wb') as wiki_fobj:
    wiki_fobj.write(etree.tostring(root, pretty_print=True))
    #print(etree.tostring(root, pretty_print=True).decode('utf8'))

Save and create xml zipped file


In [5]:
def split_fields(elem):
    title = elem.find('.//dfndum').text 
    section = elem.get('name')
    defin = elem.find('.//stmnt').text
    return (title, section, defin)

wiki = []
with gzip.open('/media/hd1/wikipedia/wiki_definitions_improved.xml.gz', 'r') as xml_fobj:
    def_xml = etree.parse(xml_fobj)
    for art in def_xml.findall('definition'):
        data = (art.find('.//dfndum').text, '', art.find('.//stmnt').text)
        wiki.append(data)
        
#random.sample(wiki, 1)
        
plmath = []
with gzip.open('/media/hd1/planetmath/datasets/planetmath_definitions.xml.gz', 'r') as xml_fobj:
    def_xml = etree.parse(xml_fobj)
    for art in def_xml.findall('article'):
        plmath.append(split_fields(art))
stacks = []
with gzip.open('/media/hd1/stacks-project/datasets/stacks-definitions.xml.gz', 'r') as xml_fobj:
    def_xml = etree.parse(xml_fobj)
    for art in def_xml.findall('article'):
        try:
            stacks.append(split_fields(art))
        except AttributeError:
            print('The name of the problematic article is: {}'.format(art.attrib['name']))

# Print results
print('Found {} in the wiki dataset.'.format(len(wiki)))
print('Found {} in the stacks dataset.'.format(len(stacks)))
print('Found {} in the PlanetMath dataset.'.format(len(plmath)))

The name of the problematic article is: examples.xml
The name of the problematic article is: coding.xml
The name of the problematic article is: spaces-pushouts.xml
The name of the problematic article is: guide.xml
The name of the problematic article is: moduli.xml
The name of the problematic article is: more-groupoids.xml
The name of the problematic article is: chapters.xml
The name of the problematic article is: sets.xml
The name of the problematic article is: obsolete.xml
The name of the problematic article is: examples-defos.xml
The name of the problematic article is: spaces-more-cohomology.xml
The name of the problematic article is: bibliography.xml
The name of the problematic article is: fdl.xml
The name of the problematic article is: limits.xml
The name of the problematic article is: conventions.xml
The name of the problematic article is: introduction.xml
The name of the problematic article is: quot.xml
The name of the problematic article is: desirables.xml
Found 5314 in the wiki

In [6]:
# Get data and train the Sentence tokenizer
# Uses a standard algorithm (Kiss-Strunk) for unsupervised sentence boundary detection
text = ''
for i in range(550):
    #text += unwiki.loads(eval(wiki[i].split('-#-%-')[2]))
    text += plmath[i][2]
for i in range(50):
    text += stacks[i][2]
for i in range(550):
    try:
        text += wiki[i][2]
    except TypeError:
        print(wiki[i])

trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
tokenizer = PunktSentenceTokenizer(trainer.get_params())
print(tokenizer._params.abbrev_types)

{'jie', 'jr', 'i.o', 's.t', '2p', 'mr', 'cf', 'n.b', 'wings', 'a.k.a', 'etc', 'r.a', 's^2', 'pp', 'al', 'axe', 'mixture', 'hk', 'i.e', 'vol', 'resp', 'az', 'scand', 'x+2', 'spacewalks', 'j.w', 'eng', 'ginebra', 'e.g', 'pl', 'p.h.d', 'forever—e.g', '62f', 'inc', 'ca', 'j.a', 'vibrations', 'ton', 'brethren', 'og', '15k', 'eqs', 'juniper', 'hyp', 'u.n', 'eq', 'u.s', 'sow', 'z-1', 'dr', 'sgs', 'ex', 'a.e', 'fig', 'missions', 'c.c.c'}


In [7]:
# The cleaning up of the wiki markup so that it looks like normal written english
title, section, defin = plmath[850]
print(title)
print(section)
defin

anti self-dual
15A63-Selfdual.xml


' Let _inline_math_ be a finite-dimensional inner-product space over a field _inline_math_. Let _inline_math_ be an endomorphism, and note that the adjoint endomorphism _inline_math_ is also an endomorphism of _inline_math_. It is therefore possible to add, subtract, and compare _inline_math_ and _inline_math_, and we are able to make the following definitions. An endomorphism _inline_math_ is said to be self-dual (a.k.a. self-adjoint) if _display_math_ By contrast, we say that the endomorphism is anti self-dual if _display_math_  Exactly the same definitions can be made for an endomorphism of a complex vector space with a Hermitian inner product.  All of these definitions have their counterparts in the matrix setting. Let _inline_math_ be the matrix of _inline_math_ relative to an orthogonal basis of _inline_math_. Then _inline_math_ is self-dual if and only if _inline_math_ is a symmetric matrix, and anti self-dual if and only if _inline_math_ is a skew-symmetric matrix.  In the case

In [8]:
def_lst = ner.bio_tag.put_pos_ner_tags(stacks, tokenizer)\
         + ner.bio_tag.put_pos_ner_tags(plmath, tokenizer)\
         + ner.bio_tag.put_pos_ner_tags(wiki, tokenizer)

In [10]:
random.shuffle(def_lst)
training_samples = [d['ner'] for d in def_lst[:int(len(def_lst) * 0.8)]]
test_samples = [d['ner'] for d in def_lst[int(len(def_lst) * 0.8):]]
 
print("#training samples = %s" % len(training_samples) )   # training samples = 55809
print("#test samples = %s" % len(test_samples))            # test samples = 6201

#training samples = 20880
#test samples = 5220


In [11]:
#train the NER Chunking Classifier (TAKES A LONG TIME)
#%time chunker = NamedEntityChunker(training_samples)
classfier = SklearnClassifier(LinearSVC())
%time chunker = NamedEntityChunker(training_samples, classifier_builder=classfier.train)

CPU times: user 1min 4s, sys: 524 ms, total: 1min 5s
Wall time: 1min 5s


In [33]:
# Evaluate the most common metrics on the test dataset
unpack = lambda l: [(tok, pos, ner) for ((tok, pos), ner) in l]
Tree_lst = [conlltags2tree(unpack(t)) for t in test_samples]
print(chunker.evaluate(Tree_lst))

ChunkParse score:
    IOB Accuracy:  94.1%%
    Precision:     69.1%%
    Recall:        11.7%%
    F-Measure:     20.0%%


In [14]:
print(Tree_lst[13].leaves())
print(chunker.parse(Tree_lst[3].leaves()))

[('A', 'DT'), ('snake-arm', 'JJ'), ('robot', 'NN'), ('is', 'VBZ'), ('not', 'RB'), ('to', 'TO'), ('be', 'VB'), ('confused', 'VBN'), ('with', 'IN'), ('a', 'DT'), ('snakebot', 'NN'), ('which', 'WDT'), ('mimics', 'VBZ'), ('the', 'DT'), ('biomorphic', 'JJ'), ('motion', 'NN'), ('of', 'IN'), ('a', 'DT'), ('snake', 'NN'), ('in', 'IN'), ('order', 'NN'), ('to', 'TO'), ('slither', 'VB'), ('along', 'IN'), ('the', 'DT'), ('ground', 'NN'), ('.', '.')]
(S
  In/IN
  the/DT
  In/IN
  1988/CD
  ,/,
  Gleason/NNP
  showed/VBD
  that/IN
  an/DT
  _inline_math_-sided/JJ
  regular/JJ
  polygon/NN
  can/MD
  be/VB
  constructed/VBN
  with/IN
  ruler/NN
  and/CC
  compass/NN
  if/IN
  _inline_math_/NN
  is/VBZ
  the/DT
  product/NN
  of/IN
  two/CD
  Pierpont/NNP
  primes/NNS
  ./.)


### Other Scores

Training with an amount of the dataset and evaluating with the rest
* With 80% split of the stacks data:
```
ChunkParse score:
    IOB Accuracy:  86.5%%
    Precision:     29.0%%
    Recall:        60.0%%
    F-Measure:     39.1%%
```

* With 80% of Wiki + Planetmath data:
```
ChunkParse score:
    IOB Accuracy:  88.4%%
    Precision:     26.5%%
    Recall:        66.3%%
    F-Measure:     37.9%%
```

* With 90% of Wiki + Planetmath:
```
ChunkParse score:
    IOB Accuracy:  88.7%%
    Precision:     27.1%%
    Recall:        67.7%%
    F-Measure:     38.7%%
```

* With 90% of the dataset Planetmath data alone:
```
ChunkParse score:
    IOB Accuracy:  86.0%%
    Precision:     24.0%%
    Recall:        67.3%%
    F-Measure:     35.3%%
```
    
* With 80% of the dataset

* 60% of the data

```
ChunkParse score:
    IOB Accuracy:  91.0%%
    Precision:     30.7%%
    Recall:        63.9%%
    F-Measure:     41.5%%
```
    
```
ChunkParse score:
    IOB Accuracy:  90.6%%
    Precision:     32.4%%
    Recall:        68.7%%
    F-Measure:     44.0%%
```

* 90% of the data

```
ChunkParse score:
    IOB Accuracy:  91.2%%
    Precision:     32.0%%
    Recall:        68.0%%
    F-Measure:     43.5%%
```
    


In [15]:
# An example of a user fed definition
print(chunker.parse(pos_tag(word_tokenize("We define a Banach space as a complete vector space."))))

(S
  We/PRP
  define/VBP
  a/DT
  (DFNDUM Banach/NNP space/NN)
  as/IN
  a/DT
  complete/JJ
  vector/NN
  space/NN
  ./.)


In [16]:
def prepare_for_metrics(int_range, chunker_fn, data_set = test_samples, print_output=False):
    '''
    `int_range` is an integer range
    NEEDS A TEST_SAMPLES VARIABLE CREATED WHEN SPLITTING THE 
    TRAINING AND TESTING DATA
    Returns two vectors ready to be used in the 
    metrics classification function
    '''
    if isinstance(int_range, int):
        int_range = [int_range]
    y_true = []
    y_pred = []
    for i in int_range:
        sample = data_set[i]
        sm = [s[0] for s in sample]
        y_true_tmp = [s[1] for s in sample]
        predicted = [v[2] for v in tree2conlltags(chunker_fn.parse(sm))]
        y_true += y_true_tmp
        y_pred += predicted
        if print_output:
            for k,s in enumerate(sm):
                print('{:15} {:>10}  {:>10}'.format(s[0], y_true_tmp[k], predicted[k]))
    return y_true, y_pred

In [17]:
OO = prepare_for_metrics(19, chunker, data_set=test_samples, print_output=True)

group                    O           O
The                      O           O
Janko             B-DFNDUM    B-DFNDUM
groups            I-DFNDUM    I-DFNDUM
denoted                  O           O
by                       O           O
_inline_math_            O           O
,                        O           O
and                      O           O
_inline_math_            O           O
are                      O           O
four                     O           O
of                       O           O
the                      O           O
26                       O           O
sporadic                 O           O
groups                   O           O
.                        O           O


In [18]:
y_true, predicted = prepare_for_metrics(range(len(test_samples)), chunker)
print(metrics.classification_report(y_true, predicted))

              precision    recall  f1-score   support

    B-DFNDUM       0.73      0.64      0.68      5873
    I-DFNDUM       0.69      0.65      0.67      4319
           O       0.98      0.98      0.98    148880

    accuracy                           0.96    159072
   macro avg       0.80      0.76      0.78    159072
weighted avg       0.96      0.96      0.96    159072



In [ ]:
from sklearn.neighbors import KNeighborsClassifier

classifiers = [
    #MultinomialNB(),
    #KNeighborsClassifier(2),
    #LinearSVC(),
    #SVC(),
    NuSVC(nu=0.01),
    #SVC(kernel="rbf", C=0.025, probability=True),
    #DecisionTreeClassifier(),
    #RandomForestClassifier(),
    #AdaBoostClassifier(),
    #GradientBoostingClassifier(),
    #GaussianNB(),
    #LinearDiscriminantAnalysis(),
    #QuadraticDiscriminantAnalysis(),
]

Tree_lst = [conlltags2tree(unpack(t)) for t in test_samples]
for k,clf in enumerate(classifiers):
    name = clf.__class__.__name__
    #clf.fit(xtrain, train_y)
    clfier = SklearnClassifier(clf)
    #clfier = clf
    Now = dt.now()
    chunker = NamedEntityChunker(training_samples, classifier_builder=clfier.train)
    training_time = (dt.now() - Now).seconds
    
    print("="*30)
    print(name)
    
    print('****Results****')
    # Evaluate the most common metrics on the test dataset
    unpack = lambda l: [(tok, pos, ner) for ((tok, pos), ner) in l]
    Now = dt.now()
    score = chunker.evaluate(Tree_lst)
    acc = score.accuracy()
    f1 = score.f_measure()
    Seconds = (dt.now() - Now).seconds
    inference_rate = len(Tree_lst)/Seconds
    print("IOB Accuracy: {:.2%}   ".format(acc))
    
    #prodictions = prob_prediction
    print("F1 Measure:   {:.2%}".format(f1))
    print(f"{Seconds = } secs")
    print(f"InferRate:   {inference_rate:.2f} sentences/sec")
    
    #log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    #log = log.append(log_entry)
    
print("="*30)

In [44]:
inference_rate

348.0

In [54]:
Seconds

41275

In [55]:
41275/60

687.9166666666666